## Gerando arquivo para NER-Genia

Teste com formato QA.. tem como retornar as entidades descontinuas tbm?

In [1]:
import os
from pathlib import Path
import re
import pickle
import random


In [2]:
def getTiposEntidade():
    return ['DNA','RNA','Cell_type','Cell_line','Protein']

In [3]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [4]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        raise
        return None


In [5]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [6]:
dic_sentencesTest=load_obj('dicEntidadesTesteGenia')
dic_sentencesDev=load_obj('dicEntidadesDevGenia')
dic_sentencesTrain=load_obj('dicEntidadesTrainGenia')

In [7]:
len(dic_sentencesTrain)

15023

In [8]:
dic_sentencesTrain[0]

[' IL - 2 gene expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
 [['IL - 2 gene', [0, 1, 2, 3], 'DNA'],
  ['NF - kappa B', [6, 7, 8, 9], 'protein'],
  ['CD28', [12], 'protein'],
  ['5 - lipoxygenase', [18, 19, 20], 'protein']]]

In [9]:
dic_sentencesTest[3]

[' Characterization of HB24 expression in lymphoid and select developing tissues was performed by in situ hybridization .',
 [['HB24', [2], 'DNA']]]

In [10]:
def arrumaDic(dic_sentences):
    dic_sentences2={}
    for key, value in dic_sentences.items():
        #print('key:', key)
        #print('value:', value)
        tokens=value[0]
        #print('tokens:', tokens)
        #listaTokens=[[t, i] for i, t in enumerate(tokens.strip().split(' '))]
        listaTokens=[]
        ultimoIndice=0
        for i, t in enumerate(tokens.strip().split(' ')):
            inicio=ultimoIndice
            listaTokens.append([t, i, inicio])
            ultimoIndice=ultimoIndice+len(t)+1

        entidades=value[1]
        valores=[listaTokens, entidades]
        dic_sentences2[key]=valores
        #break
    return dic_sentences2

dic_sentencesTest2 = arrumaDic(dic_sentencesTest)
dic_sentencesTrain2 = arrumaDic(dic_sentencesTrain)
dic_sentencesDev2 = arrumaDic(dic_sentencesDev)

dic_sentencesTrain2[0]

[[['IL', 0, 0],
  ['-', 1, 3],
  ['2', 2, 5],
  ['gene', 3, 7],
  ['expression', 4, 12],
  ['and', 5, 23],
  ['NF', 6, 27],
  ['-', 7, 30],
  ['kappa', 8, 32],
  ['B', 9, 38],
  ['activation', 10, 40],
  ['through', 11, 51],
  ['CD28', 12, 59],
  ['requires', 13, 64],
  ['reactive', 14, 73],
  ['oxygen', 15, 82],
  ['production', 16, 89],
  ['by', 17, 100],
  ['5', 18, 103],
  ['-', 19, 105],
  ['lipoxygenase', 20, 107],
  ['.', 21, 120]],
 [['IL - 2 gene', [0, 1, 2, 3], 'DNA'],
  ['NF - kappa B', [6, 7, 8, 9], 'protein'],
  ['CD28', [12], 'protein'],
  ['5 - lipoxygenase', [18, 19, 20], 'protein']]]

In [11]:
def gravaArquivoQA(dic_sentences, name):
    tipos= ['DNA','RNA','cell_type','cell_line','protein']

    texto=r'{"data": ['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesDNA=list()
        listaEntidadesRNA=list()
        listaEntidadesCell_type=list()
        listaEntidadesCell_line=list()
        listaEntidadesProtein=list()


        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'DNA':#problema
                listaEntidadesDNA.append(entidade)
            elif entidade[2] == 'RNA': #teste
                listaEntidadesRNA.append(entidade)
            elif entidade[2] == 'cell_type':#tratamento
                listaEntidadesCell_type.append(entidade)
            elif entidade[2] == 'cell_line':#anatomia
                listaEntidadesCell_line.append(entidade)
            elif entidade[2] == 'protein':
                listaEntidadesProtein.append(entidade)

        if len(listaEntidadesDNA)==0 and len(listaEntidadesRNA)==0 and len(listaEntidadesCell_type)==0 and len(listaEntidadesCell_line)==0 and len(listaEntidadesProtein)==0:
            continue

        title='texto_'+str(i)
        if num==0:
            texto = texto+'{"title":"'+title+'","paragraphs": [{"context": "'
        else:
            texto = texto[:len(texto)-1]
            texto = texto+']}]}, {"title":"'+title+'","paragraphs": [{"context": "'
        
        num=num+1

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        #contexto = contexto.replace(' , ', ', ').replace(' : ', ': ').replace(' . ', '. ').replace(' / ', '/ ')
        if '"' in contexto:
            print('tem aspassssss')
        texto = texto+contexto+'", "qas": ['


        if len(listaEntidadesDNA)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesDNA:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'
                

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "DNA", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesRNA)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesRNA:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [j[2] for j in dic_sentences[i][0] if j[1]==start][0]
                except:
                    print()
                    raise
                fim_string = [j[2] for j in dic_sentences[i][0] if j[1]==fim][0]
                fim_string = fim_string+[len(j[0]) for j in dic_sentences[i][0] if j[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "RNA", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesCell_type)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesCell_type:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                except:
                    print('i:', i)
                    print('dic_sentences[i][0]:', dic_sentences[i][0])
                    raise
                
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Cell_type", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesCell_line)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesCell_line:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                except:
                    print('i:', i)
                    print('dic_sentences[i][0]:', dic_sentences[i][0])
                    raise
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            # tirando ultima virgula
            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Cell_line", "id":"'+str(idQa)+'"},'
			
        if len(listaEntidadesProtein)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesProtein:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                except:
                    print('i:', i)
                    print('dic_sentences[i][0]:', dic_sentences[i][0])
                    raise                
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            # tirando ultima virgula
            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Protein", "id":"'+str(idQa)+'"},'			

    texto = texto[:len(texto)-1]
    texto = texto+']}]}], "version": "1.1"}'   
    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA(dic_sentencesTest2, 'data_qa/genia_test.json')
gravaArquivoQA(dic_sentencesTrain2, 'data_qa/genia_train.json')
gravaArquivoQA(dic_sentencesDev2, 'data_qa/genia_dev.json')

tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss22
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss


KeyboardInterrupt: 

In [ ]:
dic_sentencesTrain2[13374]

In [ ]:
dic_sentencesTrain2[9554]

In [ ]:
dic_sentencesTrain2[5865]

In [ ]:
dic_sentencesTest2[1]

In [179]:
import json 

try:
  file = r'data_qa/genia_train.json'
  # Opening JSON file & returns JSON object as a dictionary 
  #f = open(file, encoding="utf8") 
  f = open(file, encoding="utf8") 
  data = json.load(f) 
except:
  print('nao abriu arquivo:', file)
  print(data)
  raise

In [39]:
# versão 2 do squad
def gravaArquivoQA_impossible(dic_sentences, name):
    print('name:', name)
    tipos= ['DNA','RNA','Cell_type','Cell_line','Protein']

    texto=r'{"version": "v2.0", "data": ['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesDNA=list()
        listaEntidadesRNA=list()
        listaEntidadesCell_type=list()
        listaEntidadesCell_line=list()
        listaEntidadesProtein=list()


        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'DNA':#problema
                listaEntidadesDNA.append(entidade)
            elif entidade[2] == 'RNA': #teste
                listaEntidadesRNA.append(entidade)
            elif entidade[2] == 'cell_type':#tratamento
                listaEntidadesCell_type.append(entidade)
            elif entidade[2] == 'cell_line':#anatomia
                listaEntidadesCell_line.append(entidade)
            elif entidade[2] == 'protein':
                listaEntidadesProtein.append(entidade)

        if len(listaEntidadesDNA)==0 and len(listaEntidadesRNA)==0 and len(listaEntidadesCell_type)==0 and len(listaEntidadesCell_line)==0 and len(listaEntidadesProtein)==0:
            continue

        title='texto_'+str(i)
        if num==0:
            texto = texto+'{"title":"'+title+'","paragraphs": [{"qas": ['
        else:
            texto = texto[:len(texto)-1]
            texto = texto+'}]}, {"title":"'+title+'","paragraphs": [{"qas": ['
        
        num=num+1

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        
        
        if len(listaEntidadesDNA)>0:
            if '"' in entidade[0]:
                print('tem aspas em DNA')
            idQa=idQa+1
            texto = texto+'{"question": "DNA", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesDNA:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "DNA", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesRNA)>0:
            if '"' in entidade[0]:
                print('tem aspas em RNA')
            idQa=idQa+1
            texto = texto+'{"question": "RNA", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesRNA:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "RNA", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesCell_type)>0:
            if '"' in entidade[0]:
                print('tem aspas Cell_type')
            idQa=idQa+1
            texto = texto+'{"question": "Cell_type", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesCell_type:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Cell_type", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesCell_line)>0:
            if '"' in entidade[0]:
                print('tem aspas em Cell_line')
            idQa=idQa+1
            texto = texto+'{"question": "Cell_line", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesCell_line:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Cell_line", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'   
        
        if len(listaEntidadesProtein)>0:
            if '"' in entidade[0]:
                print('tem aspas em Protein')
            idQa=idQa+1
            texto = texto+'{"question": "Protein", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesProtein:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Protein", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'
            
        texto = texto[:len(texto)-1]
        texto = texto+'], "context":"'+contexto+'"}' 
        
    texto = texto+']}]}'

    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA_impossible(dic_sentencesTest2, 'data_qa_impossible/genia_test_impossible.json')
gravaArquivoQA_impossible(dic_sentencesTrain2, 'data_qa_impossible/genia_train_impossible.json')
gravaArquivoQA_impossible(dic_sentencesDev2, 'data_qa_impossible/genia_dev_impossible.json')

name: data_qa_impossible/genia_test_impossible.json
name: data_qa_impossible/genia_train_impossible.json
tem aspas em Cell_line
tem aspas em DNA
tem aspas em Protein
tem aspas em Protein
name: data_qa_impossible/genia_dev_impossible.json
tem aspas em Protein


In [14]:
dic_sentencesTest2[0]

[[['There', 0, 0],
  ['is', 1, 6],
  ['a', 2, 9],
  ['single', 3, 11],
  ['methionine', 4, 18],
  ['codon', 5, 29],
  ['-', 6, 35],
  ['initiated', 7, 37],
  ['open', 8, 47],
  ['reading', 9, 52],
  ['frame', 10, 60],
  ['of', 11, 66],
  ['1', 12, 69],
  [',', 13, 71],
  ['458', 14, 73],
  ['nt', 15, 77],
  ['in', 16, 80],
  ['frame', 17, 83],
  ['with', 18, 89],
  ['a', 19, 94],
  ['homeobox', 20, 96],
  ['and', 21, 105],
  ['a', 22, 109],
  ['CAX', 23, 111],
  ['repeat', 24, 115],
  [',', 25, 122],
  ['and', 26, 124],
  ['the', 27, 128],
  ['open', 28, 132],
  ['reading', 29, 137],
  ['frame', 30, 145],
  ['is', 31, 151],
  ['predicted', 32, 154],
  ['to', 33, 164],
  ['encode', 34, 167],
  ['a', 35, 174],
  ['protein', 36, 176],
  ['of', 37, 184],
  ['51', 38, 187],
  [',', 39, 190],
  ['659', 40, 192],
  ['daltons', 41, 196],
  ['.', 42, 204]],
 [['methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  ['homeobox', [20], 'DNA'],
  ['CAX repeat', [

In [13]:
dic_sentencesDev2[0]

[[['The', 0, 0],
  ['inhibition', 1, 4],
  ['of', 2, 15],
  ['c', 3, 18],
  ['-', 4, 20],
  ['fos', 5, 22],
  ['and', 6, 26],
  ['c', 7, 30],
  ['-', 8, 32],
  ['jun', 9, 34],
  ['expression', 10, 38],
  ['by', 11, 49],
  ['IL', 12, 52],
  ['-', 13, 55],
  ['4', 14, 57],
  ['in', 15, 59],
  ['LPS', 16, 62],
  ['-', 17, 66],
  ['treated', 18, 68],
  ['cells', 19, 76],
  ['was', 20, 82],
  ['shown', 21, 86],
  ['to', 22, 92],
  ['be', 23, 95],
  ['due', 24, 98],
  ['to', 25, 102],
  ['a', 26, 105],
  ['lower', 27, 107],
  ['transcription', 28, 113],
  ['rate', 29, 127],
  ['of', 30, 132],
  ['the', 31, 135],
  ['c', 32, 139],
  ['-', 33, 141],
  ['fos', 34, 143],
  ['and', 35, 147],
  ['c', 36, 151],
  ['-', 37, 153],
  ['jun', 38, 155],
  ['genes', 39, 159],
  ['.', 40, 165]],
 [['c - fos', [3, 4, 5], 'DNA'],
  ['c - fos', [32, 33, 34], 'DNA'],
  ['c - jun genes', [36, 37, 38, 39], 'DNA'],
  ['LPS - treated cells', [16, 17, 18, 19], 'cell_type'],
  ['IL - 4', [12, 13, 14], 'protein']]]

In [18]:
#len(dic_sentencesTrain2)
dic_sentencesTrain2[15022]

[[['When', 0, 0],
  ['cells', 1, 5],
  ['were', 2, 11],
  ['treated', 3, 16],
  ['with', 4, 24],
  ['IL', 5, 29],
  ['-', 6, 32],
  ['4', 7, 34],
  ['for', 8, 36],
  ['5', 9, 40],
  ['hours', 10, 42],
  ['before', 11, 48],
  ['LPS', 12, 55],
  ['activation', 13, 59],
  [',', 14, 70],
  ['both', 15, 72],
  ['the', 16, 77],
  ['c', 17, 81],
  ['-', 18, 83],
  ['fos', 19, 85],
  ['and', 20, 89],
  ['the', 21, 93],
  ['c', 22, 97],
  ['-', 23, 99],
  ['jun', 24, 101],
  ['mRNA', 25, 105],
  ['expression', 26, 110],
  ['was', 27, 121],
  ['decreased', 28, 125],
  ['.', 29, 135]],
 [['IL - 4', [5, 6, 7], 'protein']]]

# gerando agora no formato do MRC

In [ ]:
# versão MRC

def gravaArquivoQA_mrc(dic_sentences, name):
    print('name:', name)
    tipos= ['DNA','RNA','Cell_type','Cell_line','Protein']


    texto=r'['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesDNA=list()
        listaEntidadesRNA=list()
        listaEntidadesCell_type=list()
        listaEntidadesCell_line=list()
        listaEntidadesProtein=list()
        

        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'DNA':#problema
                listaEntidadesDNA.append(entidade)
            elif entidade[2] == 'RNA': #teste
                listaEntidadesRNA.append(entidade)
            elif entidade[2] == 'Cell_type':#tratamento
                listaEntidadesCell_type.append(entidade)
            elif entidade[2] == 'Cell_line':#anatomia
                listaEntidadesCell_line.append(entidade)
            elif entidade[2] == 'Protein':
                listaEntidadesProtein.append(entidade)

                
                
        if len(listaEntidadesDNA)==0 and len(listaEntidadesRNA)==0 and len(listaEntidadesCell_type)==0 and len(listaEntidadesCell_line)==0 and len(listaEntidadesProtein)==0:
            continue

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        
        texto = texto+'{"context":"'+contexto+'",' 
        
        #if len(listaEntidadesProblema)>0:
        tipo='DNA'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesDNA:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            #start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            #fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            #fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        tipo='RNA'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesRNA:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        tipo='Cell_type'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesCell_type:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        tipo='Cell_line'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesCell_line:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        tipo='Protein'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesProtein:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 
       
        
    #texto = texto+']'
    texto =texto[0:-1]+ ']'

    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA_mrc(dic_sentencesTest2, 'data_qa_mrc/genia_test_mrc.json')
gravaArquivoQA_mrc(dic_sentencesTrain2, 'data_qa_mrc/genia_train_mrc.json')
gravaArquivoQA_mrc(dic_sentencesDev2, 'data_qa_mrc/genia_dev_mrc.json')